# flask webgoat

[flask-webgoat](https://github.com/HooliCorp/flask-webgoat) is a deliberately-vulnerable application written with the Flask web framework.

In [1]:
import asyncio

from joern_lib import client, workspace
from joern_lib.detectors import common as cpg, python
from joern_lib.utils import print_table, print_tree, print_flows

joern_host = "http://joern:9000"
cpggen_host = "http://cpggen:7072"
joern_username = "admin"
joern_password  = "admin"

## Generate CPG using cpggen

To improve performance, CPG for this project is generated using the cpggen server and then imported onto the joern server. The default docker-compose starts a cpggen server with the hostname `cpggen` and port `7072`.

We use the `create_cpg` api from the workspace to directly create a CPG from the git repo. This single call would handle both gen retionsoaf the cpg and import onto joern.

In [3]:
async def generate_cpg():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await workspace.create_cpg(connection, "https://github.com/HooliCorp/flask-webgoat", out_dir="/tmp/flask-webgoat/cpg_out", languages="python", project_name="flask-webgoat")
    print (res)
asyncio.run(generate_cpg())

╭─────────────────────────────────── CPGQL Query ────────────────────────────────────╮
│ os.exists(os.Path("/tmp/flask-webgoat/cpg_out/cpggen1pej691l-python-cpg.bin.zip")) │
╰────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────────────────────── CPGQL Query ────────────────────────────────────────╮
│ importCpg("/tmp/flask-webgoat/cpg_out/cpggen1pej691l-python-cpg.bin.zip", "flask-webgoat") │
╰────────────────────────────────────────────────────────────────────────────────────────────╯
╭─ CPGQL Query ─╮
│ save          │
╰───────────────╯
True


## Inspect methods

With a fresh CPG imported, one of the initial activities to perform as a threat hunter is to inspect the methods in the application. Visually inspecting the methods would help understand the custom code and the frameworks and libraries used.

joern-lib offers a convenient api called `list_methods` to simplify the inspection. You can invoke this method without arguments to receive a full list of methods or provide a regex to narrow the results.

In [5]:
async def inspect_methods():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.list_methods(connection)
    print_table(res, title="flask-webgoat methods")
asyncio.run(inspect_methods())

╭─────────────────────── CPGQL Query ────────────────────────╮
│ cpg.method.whereNot(_.name(".*<operator>.*")).toJsonPretty │
╰────────────────────────────────────────────────────────────╯
                                                                                                                                 flask-webgoat methods                                                                                                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name                       ┃ fullName                                                             ┃ signature     ┃       lineNumberEnd ┃ _label    ┃ code     ┃ isExternal      ┃     lineNumber ┃     id ┃    order ┃ filename                       

## List decoratored methods

In most Python applications, method decorators are critical in determining the route, flows, and logic. A convenient api method called `list_decorator_location` could be used to identify all decorated methods in the application.

In [7]:
async def inspect_decorated():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await python.list_decorator_location(connection, decorator="(app|bp|blueprint).*")
    print_table([r.get("node") for r in res], title="flask-webgoat decorators")
asyncio.run(inspect_decorated())

╭──────────────────────────────────────────── CPGQL Query ─────────────────────────────────────────────╮
│ cpg.call.code(".*(app|bp|blueprint).*.*").call.assignment.argument.isMethodRef.location.toJsonPretty │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯
                                                                                                                                flask-webgoat decorators                                                                                                                                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ dynamicTypeHintFullName                  ┃ code                             ┃ typeFullName           ┃      order ┃ metho

## List HTTP Routes

In [9]:
async def get_routes():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await python.list_http_routes(connection)
    print_table(res, title="HTTP Routes")
asyncio.run(get_routes())

╭────────────────────────────────── CPGQL Query ──────────────────────────────────╮
│ cpg.call.code(".*route.*").call.assignment.map(m => (m, m.method)).toJsonPretty │
╰─────────────────────────────────────────────────────────────────────────────────╯
                                                                                                                                      HTTP Routes                                                                                                                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ annotation                                                                                        ┃ name     ┃ fullName                          ┃ signature ┃ lineNumberEnd ┃ _label ┃ 

## Static configuration

Security misconfiguration in python applications happens when configuration values such as `Debug` or HTTP headers are set statically with incorrect values. These could be usually identified by looking for dictionary assignment operations with known keys as shown.

In [11]:
async def inspect_csrf_misconfig():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await python.list_dict_assignment_location(connection, key='(Access-Control-Allow-Origin|Content-Security-Policy)')
    print_table(res, title="flask-webgoat static (mis)configuration")
asyncio.run(inspect_csrf_misconfig())

╭─────────────────────────────────────────────────────────────────────── CPGQL Query ────────────────────────────────────────────────────────────────────────╮
│ cpg.call(Operators.indexAccess).code(".*(Access-Control-Allow-Origin|Content-Security-Policy).*").call.assignment.argument.isLiteral.location.toJsonPretty │
╰────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
                                                                                                                        flask-webgoat static (mis)configuration                                                                                                                         
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Detect hardcoded sensitive values

Quick and dirty way to list potential secrets and keys without performing any entropy calculation.

In [13]:
async def detect_hardcoded_sensitive_values():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.list_sensitive_literals(connection)
    print_table(res, title="flask-webgoat hardcoded sensitive values")
asyncio.run(detect_hardcoded_sensitive_values())

╭────────────────────────────────────────────────────────────────────── CPGQL Query ──────────────────────────────────────────────────────────────────────╮
│ cpg.call.assignment.where(_.argument.order(1).code("(?i).*(secret|password|token|key|admin|root).*")).argument.order(2).isLiteral.location.toJsonPretty │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
                                                                                                                        flask-webgoat hardcoded sensitive values                                                                                                                        
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# OWASP Top 10

Having mastered the basics of Joern and joern-lib, it is time to dive into detecting OWASP top 10 vulnerabilities in this project.

## A03:2021 – Injection

### SQL Injection

In [16]:
async def detect_sqli():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, 'cpg.call.code("request\\..*").inAssignment.argument.isIdentifier.filterNot(_.name.startsWith("tmp"))', 'cpg.call.code(".*query_db.*").argument.order(1)')
asyncio.run(detect_sqli())

╭─────────────────────────────────────────────────── CPGQL Query ───────────────────────────────────────────────────╮
│ def source = cpg.call.code("request\\..*").inAssignment.argument.isIdentifier.filterNot(_.name.startsWith("tmp")) │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────────── CPGQL Query ────────────────────────╮
│ def sink = cpg.call.code(".*query_db.*").argument.order(1) │
╰────────────────────────────────────────────────────────────╯
╭────────────────────────────────── CPGQL Query ──────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJsonPretty │
╰─────────────────────────────────────────────────────────────────────────────────╯
/tmp/cpggen1pej691l/flask_webgoat/users.py:39 (username, password, int(access_level))
└── /tmp/cpggen1pej691l/flask_webgoat/users.py:38 INSERT INTO user (username, password, access_level) VALUES ('%s', '%s

### Remote Code Execution

In [18]:
async def detect_rce():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, 'cpg.call.code("request\\..*").fieldAccess.argument.orderGt(0)', 'cpg.call.code(".*subprocess.run.*").argument.order(1)')
asyncio.run(detect_rce())

╭─────────────────────────────── CPGQL Query ────────────────────────────────╮
│ def source = cpg.call.code("request\\..*").fieldAccess.argument.orderGt(0) │
╰────────────────────────────────────────────────────────────────────────────╯
╭────────────────────────── CPGQL Query ───────────────────────────╮
│ def sink = cpg.call.code(".*subprocess.run.*").argument.order(1) │
╰──────────────────────────────────────────────────────────────────╯
╭────────────────────────────────── CPGQL Query ──────────────────────────────────╮
│ sink.reachableByFlows(source).map(m => (m, m.elements.location.l)).toJsonPretty │
╰─────────────────────────────────────────────────────────────────────────────────╯
/tmp/cpggen1pej691l/flask_webgoat/actions.py:39 request.args
├── /tmp/cpggen1pej691l/flask_webgoat/actions.py:39 tmp11.get("name")
└── /tmp/cpggen1pej691l/flask_webgoat/actions.py:41 ["ps aux | grep " + name + " | awk '{print $11}'"]

/tmp/cpggen1pej691l/flask_webgoat/actions.py:39 request.args
├── /tmp